In [4]:
import requests, pandas as pd

In [ ]:
r = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/paises?view=nivelado')

In [5]:
pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/estados?view=nivelado', orient='records')

,UF-id,UF-sigla,UF-nome,regiao-id,regiao-sigla,regiao-nome
0,11,RO,Rondônia,1,N,Norte
1,12,AC,Acre,1,N,Norte
2,13,AM,Amazonas,1,N,Norte
3,14,RR,Roraima,1,N,Norte
4,15,PA,Pará,1,N,Norte
5,16,AP,Amapá,1,N,Norte
6,17,TO,Tocantins,1,N,Norte
7,21,MA,Maranhão,2,NE,Nordeste
8,22,PI,Piauí,2,NE,Nordeste
9,23,CE,Ceará,2,NE,Nordeste


In [2]:
from ibge.localidades import Estados

Estados().json.

<bound method Estados.json of [{'id': 11, 'sigla': 'RO', 'nome': 'Rondônia', 'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}, {'id': 12, 'sigla': 'AC', 'nome': 'Acre', 'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}, {'id': 13, 'sigla': 'AM', 'nome': 'Amazonas', 'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}, {'id': 14, 'sigla': 'RR', 'nome': 'Roraima', 'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}, {'id': 15, 'sigla': 'PA', 'nome': 'Pará', 'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}, {'id': 16, 'sigla': 'AP', 'nome': 'Amapá', 'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}, {'id': 17, 'sigla': 'TO', 'nome': 'Tocantins', 'regiao': {'id': 1, 'sigla': 'N', 'nome': 'Norte'}}, {'id': 21, 'sigla': 'MA', 'nome': 'Maranhão', 'regiao': {'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}}, {'id': 22, 'sigla': 'PI', 'nome': 'Piauí', 'regiao': {'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}}, {'id': 23, 'sigla': 'CE', 'nome': 'Ceará', 'regiao': {'id': 2, 'sigla': 'NE', 'nome'

In [ ]:
pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/municipios?view=nivelado', orient='records')

In [7]:
import pandas as pd
import dags.DW_TOOLS as dwt

con = dwt.create_connection(
    server='10.3.152.103',
    database='saude_mental',
    username='postgres',
    password='postgres',
    port=5432)

conn = con.connect()

In [8]:
query = """
    SELECT DISTINCT
        null as sk_municipio
        , stg."municipio-id" as cd_municipio
        , stg."municipio-nome" as no_municipio
        , stg."microrregiao-id" as cd_microrregiao
        , stg."microrregiao-nome" as no_microrregiao
        , stg."mesorregiao-id" as cd_mesorregiao
        , stg."mesorregiao-nome" as no_mesorregiao
        , stg."regiao-imediata-id" as cd_regiao_imediata
        , stg."regiao-imediata-nome" as no_regiao_imediata
        , stg."regiao-intermediaria-id" as cd_regiao_intermediaria
        , stg."regiao-intermediaria-nome" as no_regiao_intermediaria
        , stg."uf-id" as cd_uf
        , stg."uf-sigla" as ds_sigla_uf
        , stg."uf-nome" as no_uf
        , stg."regiao-id" as cd_regiao
        , stg."regiao-sigla" as ds_sigla_regiao
        , stg."regiao-nome" as no_regiao
    FROM stg.stg_municipios stg
    LEFT JOIN dw.d_municipio dim
        ON (stg."municipio-id" = dim.cd_municipio)
    WHERE dim.cd_municipio IS NULL
"""

df = dwt.read_table_from_sql(query, con)

In [10]:
df[df['cd_municipio'] == 90]

,sk_municipio,cd_municipio,no_municipio,cd_microrregiao,no_microrregiao,cd_mesorregiao,no_mesorregiao,cd_regiao_imediata,no_regiao_imediata,cd_regiao_intermediaria,no_regiao_intermediaria,cd_uf,ds_sigla_uf,no_uf,cd_regiao,ds_sigla_regiao,no_regiao


In [44]:
query = f"""
    SELECT DISTINCT
        sp."pais-m49" AS cd_naturalidade,
        sp."pais-nome" AS ds_naturalidade,
        'PAÍS' AS ds_nivel
    FROM stg.stg_pais sp
    UNION
    SELECT DISTINCT
        (su."uf-id" + 800) AS cd_naturalidade,
        su."uf-nome" AS ds_naturalidade,
        'UF' AS ds_nivel
    FROM stg.stg_uf su
"""

df = dwt.read_table_from_sql(query, con)
df

,cd_naturalidade,ds_naturalidade,ds_nivel
0,31,Azerbaijão,PAÍS
1,112,Belarus,PAÍS
2,824,Rio Grande do Norte,UF
3,368,Iraque,PAÍS
4,626,Timor-Leste,PAÍS
...,...,...,...
215,72,Botsuana,PAÍS
216,208,Dinamarca,PAÍS
217,262,Djibouti,PAÍS
218,584,Ilhas Marshall,PAÍS


In [34]:
df = df.astype({"cd_naturalidade": "Int64"})
duplicates = df[df['cd_naturalidade'].duplicated()]['cd_naturalidade']

In [47]:
df = df.query("not cd_naturalidade.duplicated() or ds_nivel != 'PAÍS'")
df

,cd_naturalidade,ds_naturalidade,ds_nivel
0,31,Azerbaijão,PAÍS
1,112,Belarus,PAÍS
2,824,Rio Grande do Norte,UF
3,368,Iraque,PAÍS
4,626,Timor-Leste,PAÍS
...,...,...,...
215,72,Botsuana,PAÍS
216,208,Dinamarca,PAÍS
217,262,Djibouti,PAÍS
218,584,Ilhas Marshall,PAÍS
